In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll


In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
#query para filtrar solo las compañias relacionadas de alguna manera con design y que son posteriores a 2009.
#solamente devuelve los valores que me interesan: name, category, location, founded year e id.
new_companies = list(coll.find({'$and':
                           [ { "offices.latitude": {"$exists": True,"$ne":None} },
                            { "offices.longitude": {"$exists": True,"$ne":None} },
                            {'founded_year': {'$gt': 2009 }},
                            {'deadpooled_year':None},
                            {'$or':[{'category_code': 'advertising' },
                                    {'category_code': 'mobile' },
                                    {'category_code': 'web' },
                                    {'category_code': 'games_video' }]}]}, 
                          {'name':1,'founded_year': 1, '_id': 0,'offices.latitude':1, 
                           'offices.longitude':1, 'category_code':1, 'location':1 }))

In [5]:
print(len(new_companies))
new_companies[0]

29


{'category_code': 'web',
 'founded_year': 2011,
 'name': 'Mokitown',
 'offices': [{'latitude': 37.09024, 'longitude': -95.712891}]}

In [6]:
#funcion para sacar las coordenadas de las queries. Solamente saca la primera de las oficinas.
def getLocation(company):
    longitude = company['offices'][0]['longitude']
    latitude = company['offices'][0]['latitude']
    coordinates = [float(longitude), float(latitude)]
    
    return coordinates

In [10]:
getLocation(new_companies[0]) #probando la función en una company.

[-95.712891, 37.09024]

In [8]:
#funcion para crear df de la query con las coordenadas
def createDfFromMongo(query):
    df = pd.DataFrame(query)
    coords=[{'type': 'Point', 'coordinates':getLocation(q)}for q in query]
    df['office_coords']=coords
    df=df[['category_code','founded_year','name','office_coords']]
    return df
    

In [11]:
df= createDfFromMongo(new_companies)

In [12]:
df.head()

,category_code,founded_year,name,office_coords
0,web,2011,Mokitown,"{'type': 'Point', 'coordinates': [-95.712891, ..."
1,mobile,2010,GENWI,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,web,2013,Fixya,"{'type': 'Point', 'coordinates': [-122.323895,..."
3,games_video,2011,Social Gaming Network,"{'type': 'Point', 'coordinates': [-122.161523,..."
4,mobile,2012,Skydeck,"{'type': 'Point', 'coordinates': [-122.3254705..."


In [8]:
df.to_json('df_new_compannies.json', orient='records') #guardando el df en json para Mongo.

In [13]:
#query para filtrar las compañias posteriores a 2009.
old_companies = list(coll.find({'$and':
                           [ { "offices.latitude": {"$exists": True,"$ne":None} },
                            { "offices.longitude": {"$exists": True,"$ne":None} },
                            {'founded_year': {'$lt': 2009 }},
                            {'deadpooled_year':None}]}, 
                          {'name':1,'founded_year': 1, '_id': 0,'offices.latitude':1, 
                           'offices.longitude':1, 'category_code':1, 'location':1 }))

In [10]:
len(old_companies)

In [14]:
df_old= createDfFromMongo(old_companies)

In [15]:
df_old.head()

,category_code,founded_year,name,office_coords
0,web,2001,TechnologyGuide,"{'type': 'Point', 'coordinates': [-84.351532, ..."
1,news,2004,Digg,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,social,2004,Facebook,"{'type': 'Point', 'coordinates': [-122.151801,..."
3,web,1999,Postini,"{'type': 'Point', 'coordinates': [-122.247573,..."
4,web,2006,Geni,"{'type': 'Point', 'coordinates': [-118.393064,..."


In [13]:
df_oldcomp.to_json('df_old_companies.json', orient='records')

In [24]:
#esto una vez hemos añadido las coordenadas, para ver la distancia.

In [25]:
db, coll = connectCollection('companies','old_companies')

In [26]:
old_comp_paloalto = coll.find(
    {"office_coords":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[-122.1722525,37.4374154]
       },
       "$minDistance":1000,
       "$maxDistance":6000
      }
     }
    }
)
old_comp_paloalto=list(old_comp_paloalto)

In [27]:
print(len(old_comp_paloalto))

126


In [28]:
import folium

In [29]:
map_city=folium.Map(location=[37.4374154,-122.1722525], zoom_start=12)
for c in old_comp_paloalto:
    folium.Marker(c['office_coords']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)



In [30]:
map_city

